In [2]:
import pandas as pd
import numpy as np
from patsy import dmatrices
import json

pd.options.display.max_rows = 999
pd.options.display.max_columns = 110

In [3]:
file_path = 'collegefb/combine_update_copy.json'
with open(file_path) as json_data:
    d = json.load(json_data)

    data = []
for row in d:
    for k in row:
        data.append(row[k])
labels = ['year','player','pos','college','height','weight','forty','vertical','bench','broad','threecone','shuttle','drafted','url']

df = pd.DataFrame(data,columns=labels)

df["height"]= (df.height.str.split('-').str[0]).astype(int)*12 + (df.height.str.split('-').str[1]).astype(int)
df = df.convert_objects(convert_numeric=True)
for col in ["height","weight","forty", "vertical", "bench", "broad", "threecone", "shuttle"]:
    df[col] = df.groupby("pos")[col].transform(lambda x: x.fillna(x.median()))
    df[col] = df.groupby("pos")[col].apply(lambda x: (x-x.min())/(x.max()-x.min()))

df = df.convert_objects(convert_numeric=True)

/Users/stephenchou/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:15: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/Users/stephenchou/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:27: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [5]:
file_path = 'collegefb/draft_update2.json'
with open(file_path) as json_data:
    d = json.load(json_data)

    data = []
for row in d:
    for k in row:
        data.append(row[k])
labels = ['year','round','pick','team','player','pos','age','college','url']

df_draft = pd.DataFrame(data,columns=labels)
df_draft = df_draft.convert_objects(convert_numeric=True)
df_draft["firstround"]=df_draft['round']==1
df_draft = df_draft[['round','pick','player','year','firstround']]

/Users/stephenchou/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:13: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [7]:
file_path = 'collegefb/college_update2.json'
with open(file_path) as json_data:
    d = json.load(json_data)

    data = []
for row in d:
    for k in row:
        data.append(row[k])
#labels = ['completions','attempts','pass_yards','pass_tds','rec_yards','rec_td','receptions', 'rush_att', 'rush_yds', 'rush_td', 'solo_tackes', 'tackles', 'loss_tackles', 'ast_tackles', 'fum_forced', 'sacks', 'int', 'pd']
labels = ['player_name', 'receptions', 'rec_yards', 'rec_td', 'rush_att', 'rush_yds', 'rush_td', 'attempts', 'completions', 'pass_ints', 'pass_tds', 'pass_yards','games', 'solo_tackles', 'ast_tackles', 'tackles', 'loss_tackles', 'sacks', 'int', 'int_tds', 'int_yds', 'pd', 'fum_rec', 'fum_forced', 'fum_tds', 'fum_yds','url']

df_college = pd.DataFrame(data,columns=labels)
df_college = df_college.convert_objects(convert_numeric=True)
df_college_norm=pd.DataFrame(df_college['url'])

cols = ['receptions', 'rec_yards', 'rec_td', 'rush_att', 'rush_yds', 'rush_td', 'attempts', 'completions', 'pass_ints', 'pass_tds', 'pass_yards','games', 'solo_tackles', 'ast_tackles', 'tackles', 'loss_tackles', 'sacks', 'int', 'int_tds', 'int_yds', 'pd', 'fum_rec', 'fum_forced', 'fum_tds', 'fum_yds']

for col in cols:
    df_college_norm[col] = df_college[col]/(df_college[col].max())

df_college = df_college.drop('player_name',1)

/Users/stephenchou/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:14: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


# rankings

In [11]:
file_path = 'collegefb/cfb_rank.json'
with open(file_path) as json_data:
    d = json.load(json_data)

    data = []
for row in d:
    for k in row:
        data.append(row[k])
labels = ['poll','rank','college']

df_rank = pd.DataFrame(data,columns=labels)
df_rank['year']=df_rank.poll.str.split(' ').str[0].astype(int)+1
df_rank['college_r']=1

# Combine datasets

In [13]:
temp_df = df.merge(df_rank,how='left',on=['college','year'])
temp_df = temp_df.merge(df_draft,how='left',on=['year','player'])
complete_df = temp_df.merge(df_college,how='left',on=['url'])
complete_df_norm = temp_df.merge(df_college_norm,how='left',on=['url'])
complete_df = complete_df.convert_objects(convert_numeric=True)
complete_df_norm = complete_df_norm.convert_objects(convert_numeric=True)
complete_df.fillna(0, inplace=True)
complete_df_norm.fillna(0, inplace=True)

/Users/stephenchou/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:12: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/Users/stephenchou/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:13: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [15]:
complete_df_norm.to_pickle('merged_football_data_norm_update.pkl')
complete_df.to_pickle('merged_football_data_update.pkl')